In [1]:
import pandas as pd


In [2]:
stopsData1 = pd.read_excel("stopsData1_0319.xlsx")
stopsData2 = pd.read_excel("stopsData2_0319.xlsx")
df = stopsData1.merge(stopsData2,how='inner',on='stop_id')

In [35]:
df.groupby(['driver_sex','ticket'])['stop_id'].count()

driver_sex  ticket
f           0         659
            1         190
m           0         776
            1         515
Name: stop_id, dtype: int64

In [26]:
df

,stop_id,officer_id,driver_id,driver_age,driver_sex,ticket,ward,isMale
0,7406,B,3566,33,m,0,1,1.0
1,3364,A,9243,28,m,1,7,1.0
2,1934,D,1296,66,m,0,3,1.0
3,2807,E,14277,28,m,0,8,1.0
4,9974,D,2493,55,m,0,2,1.0
5,5339,A,5785,48,m,1,2,1.0
6,9579,C,21165,52,m,1,4,1.0
7,34,B,3794,60,f,0,6,0.0
8,4172,A,14186,34,f,0,1,0.0
9,6845,D,16307,51,m,1,1,1.0


In [4]:
# How many traffic stops are there in the data? 
df.shape

(2140, 7)

In [5]:
# How many different drivers were stopped? 
df.driver_id.nunique()

2034

In [6]:
# What’s the average age of those stopped? 
df.driver_age.mean()

47.11822429906542

In [7]:
# What percentage of stops are men? 
df[df.driver_sex=='m'].shape[0]/df.stop_id.nunique()

0.6032710280373832

In [8]:
# Which ward has highest average age of those stopped? 
df.groupby(['ward'])['driver_age'].mean().sort_values(ascending=False)

ward
8    50.905882
6    49.923695
7    48.432836
3    46.878788
5    46.021053
4    45.571429
2    44.869565
1    44.867188
Name: driver_age, dtype: float64

In [9]:
'''Using this model, if driver A is x years old, and driver B is x + 10 years old, 
how much more or less likely to receive a ticket is driver B than driver A?'''
import statsmodels.api as st
df['isMale']=pd.get_dummies(df.driver_sex, dtype=float)['m']
x = df.loc[:,['driver_age','isMale']]
x = st.add_constant(x, prepend = False)
y = df.ticket
mdl = st.MNLogit(y, x)

In [10]:
mdl_fit = mdl.fit()
mdl_fit.summary()

Optimization terminated successfully.
         Current function value: 0.607251
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 ticket   No. Observations:                 2140
Model:                        MNLogit   Df Residuals:                     2137
Method:                           MLE   Df Model:                            2
Date:                Fri, 22 Mar 2019   Pseudo R-squ.:                 0.04186
Time:                        02:14:29   Log-Likelihood:                -1299.5
converged:                       True   LL-Null:                       -1356.3
                                        LLR p-value:                 2.206e-25
==============================================================================
  ticket=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
driver_age     0.0185      0.003      6.282      0.000       0.013       0.024
isMale         0.8512      0.101      8.420      0.000       0.653       1.049
const         -2.1438      0.169    -12.677      0.000      -2.475      -1.812
==============================================================================
"""

Using this model, if driver A is x years old, and driver B is x + 10 years old, how much more or less likely to receive a ticket is driver B than driver A?

In [61]:
AGE = 20
driverA= mdl_fit.predict([AGE,0,1])[0][1]
driverB=mdl_fit.predict([AGE+10,0,1])[0][1]
driverB/driverA

1.1692118876476174

From the estimated model, what is the expected probability that a 30-year old female who is stopped will receive a ticket?

In [27]:
mdl_fit.predict([30,0,1])

array([[0.83024437, 0.16975563]])

From the estimated model, how much more or less likely are men to receive tickets than women of the same age?

In [55]:
AGE=0
M = mdl_fit.predict([AGE,1,1])[0][1]
F = mdl_fit.predict([AGE,0,1])[0][1]
M/F 

2.0532528753649637